In [14]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [16]:
def get_title(soup):
    try:
        title = soup.find("span", attrs= {"id": "productTitle"})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):
    try:
        price = soup.find("span", attrs = {"class" : "a-price a-text-price a-size-medium apexPriceToPay"}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs = {"class" : "a-price aok-align-center reinventPricePriceToPayMargin priceToPay"}).string.strip()
        except:
            price = ""
    return price

def get_rating(soup):
    try:
        rating = soup.find("i", attrs = {"class" : "a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big"}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs = {"class" : "a-icon-alt"}).string.strip()
        except:
            rating = ""
    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs = {"class" : "acrCustomerReviewText"}).string.strip()
    except AttributeError:
        review_count = ""

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs = {"id" : "availability"})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"

    return available

In [24]:
HEADERS = (
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
     'Accept-Language': 'en-US, en;q=0.5'
    }
)
    
URL = "https://www.amazon.com/s?k=gpu&rh=n%3A172282%2Cp_123%3A219979&dc&ds=v1%3AUINubl6ATZKgvpBCx4lnAZULJxNtIJKjA%2FubXeDDEiY&crid=3Q5RDS5K2VFXS&qid=1725529658&rnid=85457740011&sprefix=gpu%2Caps%2C351&ref=sr_nr_p_123_1"
    
webpage = requests.get(URL, headers=HEADERS)

print(webpage)
    
soup = BeautifulSoup(webpage.content, "html.parser")
    
links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    
links_list = []
    
for link in links:
    links_list.append(link.get('href'))

print(links_list)
        
data = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
for link in links_list:
    new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")
    
    print(new_soup.prettify())
        
    data['title'].append(get_title(new_soup))
    data['price'].append(get_price(new_soup))
    data['rating'].append(get_rating(new_soup))
    data['reviews'].append(get_review_count(new_soup))
    data['availability'].append(get_availability(new_soup))
        
amazon_df = pd.DataFrame.from_dict(data)
print(amazon_df.head())
amazon_df['title'] = amazon_df['title'].replace('', np.nan)
amazon_df = amazon_df.dropna(subset = ['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

<Response [503]>
[]
Empty DataFrame
Columns: [title, price, rating, reviews, availability]
Index: []


In [18]:
amazon_df.head()

,title,price,rating,reviews,availability
